# 量子门集层析

*版权所有 (c) 2022 百度量子计算研究所，保留所有权利。*

## 内容概要

回顾量子态层析（QST）和量子过程层析（QPT）两项技术，它们均假设制备的量子态或测量算符是已知的。但是，这些量子态和测量算符必须使用量子门来实现，量子门本身可能含有噪声。因此， QST 和 QPT 都存在*自一致性问题*。如果想要完整地描述一组未知的门、量子态和量子测量设备，我们可以使用量子门集层析（GateSet Tomography, GST）。

本教程将介绍量子门集层析的理论及其在[百度量子平台](https://quantum.baidu.com/)上的实践。

## 理论

### 门集

首先，我们定义什么是门集，如下式所示：

$$
    \mathcal{G} := \left\{ |\rho\rangle\rangle, \langle\langle E|, \mathsf{G}_0, \cdots, \mathsf{G}_K\right\},
$$

其中，$|\rho\rangle\rangle$ 是输入量子态 $\rho$ 的 PTM（Pauli Transfer Matrix）表达， $\langle\langle E|$ 是测量设备一个 POVM 元素的 PTM 表达， $\mathsf{G}_k$ 是量子门 $G_k$ 的 PTM 表达。这意味着，我们将所有感兴趣的信息都封装在了 $\mathcal{G}$ 中。有时候，我们仅关注量子门的含噪情况，因此也会将门集定义为如下形式：

$$
    \mathcal{G} := \left\{\mathsf{G}_0, \cdots, \mathsf{G}_K\right\}.
$$

通过结合上下文的方式，我们可以很容易分辨出两种定义。

在量子过程层析技术中，我们通过在目标量子门前后分别插入量子态制备电路和测量电路，从而获取含噪量子门 PTM 表达。同样地，量子门集层析也需要在每个目标量子门 $G_k$ 前后分别插入量子态制备电路和测量电路，我们将插入的电路定义为：

$$
    \mathcal{F} := \left\{ \mathsf{F}_1, \cdots, \mathsf{F}_N \right\}.
$$

为了保证计算结果的自一致性，我们所用的态制备电路和测量电路必须由 $G_k$ 构成，即 $\mathcal{F}$ 必须有如下形式：

$$
    \mathsf{F}_i = \mathsf{G}_{i_1} \circ \mathsf{G}_{i_2} \circ \cdots \circ \mathsf{G}_{i_{L_i}},
$$

其中 $L_i$ 是第 $i$ 个电路的长度。


### 线性求逆法

下面我们讲介绍一种简单的算法，用于获得具有自一致性的门集，我们将其称为线性求逆法。

首先，对于一个 $n$ 量子比特的系统，我们需要定义一组态制备电路和测量电路，$\mathcal{F} = \left\{\mathsf{F}_1, \cdots, \mathsf{F}_{4^n}\right\}$ 。需要注意的是，这一组电路并不能随意定义，我们需要让 $\mathcal{F}$ 作用在输入的未知量子态 $\rho$ 和量子测量设备 $E$ 后，得到的 $\{\mathsf{F}_j|\rho\rangle\rangle\}_{j=1}^{4^n}$ 和 $\{\langle\langle E|\mathsf{F}_i\}_{i=1}^{4^n}$ 能够张成整个空间。


#### 门集示例

为了能够更直观地了解门集，下面我们将示例一个简单的单比特门集，同时我们将会在[百度量子平台](https://quantum.baidu.com/)上实现该门集的层析。

* $\mathcal{G} = \left\{\left\{\right\}, X_{\pi/2}, Y_{\pi/2}\right\} = \left\{\mathsf{G}_0, \mathsf{G}_1, \mathsf{G}_2\right\}$
* $\mathcal{F} = \left\{\left\{\right\}, X_{\pi/2} \circ X_{\pi/2}, X_{\pi/2}, Y_{\pi/2}\right\} = \left\{\left\{\right\}, \mathsf{G}_1 \circ \mathsf{G}_1, \mathsf{G}_1, \mathsf{G}_2\right\}$

其中 $X_{\pi/2}$ 的作用是将量子态绕布洛赫球的 $X$ 轴旋转 $\pi/2$ 度，符号 $\{\}$ 代表空门，即不作用任何量子门。（我们始终将 $G_0$ 定义为空门，其原因将在后文解释。）


#### 数据采集

在量子门集层析中，我们处理的主要数据为：

$$
    p_{ikj} = \langle\langle E|\mathsf{F}_i\mathsf{G}_k\mathsf{F}_j|\rho\rangle\rangle,
$$

其中 $\mathsf{F}_i,\mathsf{F}_j\in\mathcal{F}$ ， $\mathsf{G}_k\in\mathcal{G}$ 。 对应的数据采集电路如下图所示。

![GST](./figures/gst-general-circuit.png "图1：我们通过构造所示电路来获取估计值。")

我们在上式中插入两组完整的 Pauli 基，得到如下式子：

$$
\begin{align}
    p_{ikj}
&=  \sum_{rs} \langle\langle E | \mathsf{F}_i | r \rangle\rangle\langle\langle r | \mathsf{G}_k | s \rangle\rangle\langle\langle s |\mathsf{F}_j | \rho \rangle\rangle \\
&=  \sum_{rs} A_{ir} \left( \mathsf{G}_k\right)_{rs} B_{sj}.
\end{align}
$$

我们可以将上式改写为如下矩阵形式：

$$
    \widetilde{\mathsf{G}}_k = A \mathsf{G}_k B,
$$

其中，$A = \sum_i | i \rangle\rangle\langle\langle E | \mathsf{F}_i $ ，$B = \sum_j \mathsf{F}_j | \rho \rangle\rangle\langle\langle j |$ ，$\left(\widetilde{\mathsf{G}}_k\right)_{ij} = p_{ikj}$ 。因此，我们的问题演变为，如何利用实验数据 $\widetilde{\mathsf{G}}_k$ 来估计 $\mathsf{G}_k$ 。

我们在前文中定义的 $\mathsf{G}_0 = \left\{\right\}$ 将会发挥作用。我们定义一个矩阵，称为 Gram 矩阵，

$$
  g := \widetilde{\mathsf{G}}_0 = A B.
$$

我们可以观察到，

$$
  g^{-1}\widetilde{\mathsf{G}}_k = B^{-1} A^{-1} A \mathsf{G}_k B = B^{-1} \mathsf{G}_k B.
$$

然后，我们定义如下矩阵：

$$
  \hat{\mathsf{G}}_k = g^{-1} \widetilde{\mathsf{G}}_k.
$$

我们可以通过对 $\hat{\mathsf{G}}_k$ 作用矩阵 $B$ 得到 $\mathsf{G}_k$ 的估计。因此，我们的问题又演变成了如何求解矩阵 $B$ 。

#### Gauge 优化

我们定义如下两个向量：

$$
\begin{align}
    | \widetilde{\rho} \rangle\rangle := A | \rho \rangle\rangle
&=  \sum_i | i\rangle\rangle \langle\langle E | \mathsf{F}_i | \rho \rangle\rangle, \\
    \langle\langle \widetilde{E} | := \langle\langle E | B
&=  \sum_j \langle\langle E | \mathsf{F}_j | \rho \rangle\rangle \langle\langle j |.
\end{align}
$$

不难发现，我们可以通过测量，获得 $| \widetilde{\rho} \rangle\rangle$ 和 $\langle\langle \widetilde{E} |$ 的估计值，并且有 $\langle\langle i|\widetilde{\rho}\rangle\rangle = \langle\langle\widetilde{E}|i\rangle\rangle = \langle\langle E|\mathsf{F}_i|\rho\rangle\rangle$ 。对应的测量电路如下图所示。

![GST](./figures/gst-circuit-new-gateset.png "图2：我们通过构造所示电路来构造新的门集。")

下面，我们将定义一个新的门集，

$$
\begin{align}
  &\hat{\mathcal{G}} := \left\{|\hat{\rho}\rangle\rangle, \langle\langle\hat{E}|, \left\{ \hat{G}_k\right\}\right\} =
  \left\{B^{-1}| \rho \rangle\rangle, \langle\langle E | B, \left\{ B^{-1} G_k B\right\}\right\}, \\
  &| \hat{\rho} \rangle\rangle := g^{-1} | \widetilde{\rho} \rangle\rangle = B^{-1} |\rho\rangle\rangle, \;
  \langle\langle \hat{E} | := \langle\langle \widetilde{E} | = \langle\langle E | B, \;
  \hat{G}_k = B^{-1} G_k B.
\end{align}
$$

对于这个新的门集，我们可以通过上述电路得到 $|\hat{\rho}\rangle\rangle$ 和 $\langle\langle\hat{E}|$ 的估计值。

然后，我们将会介绍如何通过这个新的门集来得到矩阵 $B$ 的估计。幸运的是，现有的量子设备具有较高的保真度，因此我们可以假设理想量子门与实际量子门差距并不是很大。我们可以通过计算得到理想情况下的门集为 $\mathcal{G}^t = \left\{ | \rho^t \rangle\rangle, \langle\langle E^t |, \left\{ G_k^t\right\} \right\}$ ，再结合我们实验得到的 $\hat{\mathcal{G}} = \left\{|\hat{\rho}\rangle\rangle, \langle\langle\hat{E}|, \left\{ \hat{G}_k\right\}\right\}$ ，则可以通过下式得到矩阵 $B$ 的近似估计，

$$
  B^e = \textrm{argmin}_{B} \sum_{k=0}^{K+1} \textrm{Tr} \left\{ \left( \hat{\mathsf{G}}_k - B^{-1} \mathsf{G}_k^t B \right)^T \left( \hat{\mathsf{G}}_k - B^{-1} \mathsf{G}_k^t B \right)\right\},
$$

其中我们定义 $\mathsf{G}_{K+1} := | \rho \rangle\rangle \langle\langle E |$ 。因此，我们通过下面这一系列式子得到原门集的估计，

$$
\begin{align}
  &| \rho^e \rangle\rangle = B^e|\hat{\rho}\rangle\rangle, \\
  &\langle\langle E^e | = \langle\langle \hat{E} | \left(B^e\right)^{-1}, \\
  &\mathsf{G}^e_k = B^e \hat{\mathsf{G}}_k \left(B^e\right)^{-1}.
\end{align}
$$


## 实践

下面，我们将演示如何对前文提到的门集进行量子门集层析。

首先，我们导入一些必要的包。

In [ ]:
import QCompute
import numpy as np
from Extensions.QuantumErrorProcessing.qcompute_qep.tomography import GateSetTomography, GateSet
import Extensions.QuantumErrorProcessing.qcompute_qep.tomography as tomography
import Extensions.QuantumErrorProcessing.qcompute_qep.utils.types as types
from QCompute.QPlatform.QOperation import CircuitLine

然后我们设置感兴趣的门集：

* $\mathcal{G} = \left\{\left\{\right\}, X_{\pi/2}, Y_{\pi/2}\right\} = \left\{\mathsf{G}_0, \mathsf{G}_1, \mathsf{G}_2\right\}$
* $\mathcal{F} = \left\{\left\{\right\}, X_{\pi/2}, Y_{\pi/2}, X_{\pi/2} \circ X_{\pi/2}\right\} = \left\{\left\{\right\}, \mathsf{G}_1, \mathsf{G}_2, \mathsf{G}_1 \circ \mathsf{G}_1\right\}$

QEP 中构造量子门集的方式如下所示。用户需要指定目标量子门集合，以及如何使用这些量子门去构造量子态制备与测量电路：

In [ ]:
gate_set = GateSet(gates={'G_rx90': CircuitLine(QCompute.RX(np.pi / 2), [0]),
                          'G_ry90': CircuitLine(QCompute.RY(np.pi / 2), [0])},
                          prep_gates=[['G_rx90', 'G_rx90'], ['G_rx90'], ['G_ry90']],
                          meas_gates=[['G_rx90', 'G_rx90'], ['G_rx90'], ['G_ry90']],
                          name='STD1Q_GATESET_RXRY')

QEP 内置了许多实验上常用的量子门集，用户可通过如下方式调用：

In [ ]:
# More Options: 'STD1Q_GATESET_RXRYRX', 'STD1Q_GATESET_RXRYID',
#               'STD2Q_GATESET_RXRYCX', 'STD2Q_GATESET_RXRYCZ', 'STD2Q_GATESET_RXRYSWAP',
gate_set_2 = tomography.STD1Q_GATESET_RXRY

接下来我们需要选择量子计算机的类型，可以选择模拟器或量子计算机真机，这里我们选择的是百度量子真机。剩下的步骤就很简单了，只需要调用我们的 GateSetTomography 模块则可完成真实量子门集的层析。这里的 `qubits=[1]` 意味着我们选择对第二个比特进行层析。

In [ ]:
# 使用本地模拟器
qc = QCompute.BackendName.LocalBaiduSim2

# 请登录“量易伏”平台（https://quantum-hub.baidu.com/）获取 Token
# QCompute.Define.hubToken = "Token"
# qc = QCompute.BackendName.CloudBaiduQPUQian

qc_name = types.get_qc_name(qc)

tomo = GateSetTomography()
qubits = [1]

gate_set = tomo.fit(qc=qc, gate_set=gate_set, qubits=qubits)

最后，我们将分析实验数据并进行可视化：

In [ ]:
print("*******************************************************************************")
print("GateSet Tomography Done. Information:")
print("+ GateSet name: {}".format(gate_set.name))
print("+ Quantum computer name: {}".format(types.get_qc_name(qc)))
print("+ Working qubits: {}".format(qubits))
# Show the information of the tomographic quantum gates
for name in gate_set.gate_names:
    # Get the ideal gate (in PTM representation)
    ideal = gate_set.gateset_ptm[name]
    # Get the noisy gate (in PTM representation)
    noisy = gate_set.gateset_opt[name]
    tomography.compare_process_ptm(ptms=[ideal, noisy, ideal - noisy],
                                   titles=['Ideal', qc_name, 'Difference'],
                                   show_labels=True,
                                   fig_name="GST-{}-{}.png".format(qc_name, name))
    print("+ Average gate fidelity of gate {}: {:.3f}".format(name, gate_set.fidelity(name)))

print("+ Preparation state fidelity: {:.3f}".format(gate_set.fidelity('rho')))
print("+ Computational basis measurement fidelity: {:.3f}".format(gate_set.fidelity('meas')))
print("*******************************************************************************")


至此，我们简要介绍了量子门集层析的基本理论及其在[百度量子平台](https://quantum.baidu.com/)上的实践。欢迎感兴趣的用户下载和使用 QEP，进行更深入的探索。

## 参考资料

[1] Greenbaum, Daniel. "Introduction to quantum gate set tomography." [arXiv](https://arxiv.org/abs/1509.02921) preprint arXiv:1509.02921 (2015).

[2] Nielsen, Erik, et al. "Gate Set Tomography." [Quantum](https://quantum-journal.org/papers/q-2021-10-05-557/) 5 (2021): 557.